# DES

DES is a ***block cipher*** - meaning it operates on plaintext blocks of a given size (64bits) and returns ciphertext blocks of the same size. Thus DES results in a ***permutation*** among the 2^64 (read this as: "2 to the 64th power") possible arrangements of 64 bits, each of which may be either 0 or 1.

The following process of DES Algorithm is adapted from the article [The DES Algorithm Illustrated](http://page.math.tu-berlin.de/~kant/teaching/hess/krypto-ws2006/des.htm) by J. Orlin Grabbe.

## Initial parameters

In [ ]:
# Insert 64-bit plain text here 

plain = '0123456789ABCDEF'

# Insert 64-bit key here

key = '0123456789AB3968'

# Convert to bits

plain = f'{int(plain, 16):0>64b}'
key = f'{int(key, 16):0>64b}'

In [ ]:
PC_1 = [[57,49,41,33,25,17,9],
        [1,58,50,42,34,26,18],
        [10,2,59,51,43,35,27],
        [19,11,3,60,52,44,36],
        [63,55,47,39,31,23,15],
        [7,62,54,46,38,30,22],
        [14,6,61,53,45,37,29],
        [21,13,5,28,20,12,4]]

PC_2 = [[14,17,11,24,1,5],
        [3,28,15,6,21,10],
        [23,19,12,4,26,8],
        [16,7,27,20,13,2],
        [41,52,31,37,47,55],
        [30,40,51,45,33,48],
        [44,49,39,56,34,53],
        [46,42,50,36,29,32]]

IP = [[58,50,42,34,26,18,10,2],
      [60,52,44,36,28,20,12,4],
      [62,54,46,38,30,22,14,6],
      [64,56,48,40,32,24,16,8],
      [57,49,41,33,25,17,9,1],
      [59,51,43,35,27,19,11,3],
      [61,53,45,37,29,21,13,5],
      [63,55,47,39,31,23,15,7]]

E = [[32,1,2,3,4,5],
        [4,5,6,7,8,9],
        [8,9,10,11,12,13],
        [12,13,14,15,16,17],
        [16,17,18,19,20,21],
        [20,21,22,23,24,25],
        [24,25,26,27,28,29],
        [28,29,30,31,32,1]]

shift_list = [1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,1]

In [ ]:
s_box = [[[14,4,13,1,2,15,11,8,3,10,6,12,5,9,0,7],
          [0,15,7,4,14,2,13,1,10,6,12,11,9,5,3,8],
          [4,1,14,8,13,6,2,11,15,12,9,7,3,10,5,0],
          [15,12,8,2,4,9,1,7,5,11,3,14,10,0,6,13]],
         
         [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10], 
          [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5], 
          [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15], 
          [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],

         [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8], 
          [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1], 
          [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7], 
          [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],

         [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15], 
          [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9], 
          [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4], 
          [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],

         [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9], 
          [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6], 
          [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14], 
          [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]],
            
         [[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11], 
          [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8], 
          [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6], 
          [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]],

         [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1], 
          [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6], 
          [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2], 
          [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]],

         [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7], 
          [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2], 
          [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8], 
          [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]]]

P = [[16, 7, 20, 21], 
                    [29, 12, 28, 17], 
                    [1, 15, 23, 26], 
                    [5, 18, 31, 10], 
                    [2, 8, 24, 14], 
                    [32, 27, 3, 9], 
                    [19, 13, 30, 6], 
                    [22, 11, 4, 25]]

final_per = [[40, 8, 48, 16, 56, 24, 64, 32], 
                           [39, 7, 47, 15, 55, 23, 63, 31], 
                           [38, 6, 46, 14, 54, 22, 62, 30], 
                           [37, 5, 45, 13, 53, 21, 61, 29], 
                           [36, 4, 44, 12, 52, 20, 60, 28], 
                           [35, 3, 43, 11, 51, 19, 59, 27], 
                           [34, 2, 42, 10, 50, 18, 58, 26], 
                           [33, 1, 41, 9, 49, 17, 57, 25]]

## Step 1: Create 16 subkeys, each of which is 48-bit long

Create **K+** from **K** and **PC-1** table

In [ ]:
kplus = [[key[i - 1] for i in row] for row in PC_1]

kplus_str = ''.join([''.join(row) for row in kplus])

print('K+: {}'.format(' '.join([kplus_str[i:i+7] for i in range(0, len(kplus_str), 7)])))

Split **K+** into left and right halves, **C<sub>0</sub>** and **D<sub>0</sub>**, then create sixteen blocks **C<sub>n</sub>** and **D<sub>n</sub>**. Each pair of blocks **C<sub>n</sub>** and **D<sub>n</sub>** is formed from the previous pair **C<sub>n-1</sub>** and **D<sub>n-1</sub>**, respectively, using the following schedule of "left shifts" of the previous block.

In [ ]:
c_list = [[] for _ in range(17)]
d_list = [[] for _ in range(17)]

c_list[0], d_list[0] = kplus_str[:len(kplus_str)//2], kplus_str[len(kplus_str)//2:]
print('C0: {}'.format(' '.join([c_list[0][i:i+7] for i in range(0, len(c_list[0]), 7)])))
print('D0: {}'.format(' '.join([d_list[0][i:i+7] for i in range(0, len(d_list[0]), 7)])))
print('---------------------------------')

for i in range(len((c_list[:-1]))):
    c_list[i+1] = c_list[i][shift_list[i]:] + c_list[i][:shift_list[i]]
    d_list[i+1] = d_list[i][shift_list[i]:] + d_list[i][:shift_list[i]]
    print('C{}: {}'.format(i+1, ' '.join([c_list[i+1][idx:idx+7] for idx in range(0, len(c_list[i+1]), 7)])))
    print('D{}: {}'.format(i+1, ' '.join([d_list[i+1][idx:idx+7] for idx in range(0, len(d_list[i+1]), 7)])))
    print('---------------------------------')

We now form the keys **K<sub>n</sub>**, for 1<=**n**<=16, by applying the **PC-2** permutation table to each of the concatenated pairs **C<sub>n</sub>D<sub>n</sub>**. Each pair has 56 bits, but **PC-2** only uses 48 of these.

In [ ]:
subkeys = [[] for _ in range(16)]

for i, key in enumerate(subkeys):
    key = [(c_list[i+1] + d_list[i+1])[idx - 1] for row in PC_2 for idx in row]
    key_str = ''.join(key)
    subkeys[i] = key_str
    print('K{}: {}'.format(i+1, ' '.join([key_str[idx: idx + 6] for idx in range(0, len(key_str), 6)])))

## Step 2: Encode each 64-bit block of data.

Applying the *initial permutation* **IP** to the block of text **M**.

In [ ]:
ip_str = ''.join([plain[idx - 1] for row in IP for idx in row])

print('IP: {}'.format(' '.join([ip_str[idx:idx+4] for idx in range(0, len(ip_str), 4)])))

Next divide the permuted block **IP** into a left half **L<sub>0</sub>** of 32 bits, and a right half **R<sub>0</sub>** of 32 bits.

In [ ]:
l_list = [[] for _ in range(17)]
r_list = [[] for _ in range(17)]

l_list[0], r_list[0] = ip_str[:len(ip_str)//2], ip_str[len(ip_str)//2:]
print('L0: {}'.format(' '.join([l_list[0][i:i+4] for i in range(0, len(l_list[0]), 4)])))
print('R0: {}'.format(' '.join([r_list[0][i:i+4] for i in range(0, len(r_list[0]), 4)])))
print('---------------------------------')

We now proceed through 16 iterations, for 1<=**n**<=16, using a function ***f*** which operates on two blocks--a data block of 32 bits and a key Kn of 48 bits--to produce a block of 32 bits. **Let + denote XOR addition, (bit-by-bit addition modulo 2)**. Then for **n** going from 1 to 16 we calculate

**L<sub>n</sub> = R<sub>n-1</sub>  
R<sub>n</sub> = L<sub>n-1</sub> + *f*(R<sub>n-1</sub>, K<sub>n</sub>)**  

To calculate ***f***, we first expand each block **R<sub>n-1</sub>** from 32 bits to 48 bits. This is done by using a selection table that repeats some of the bits in **R<sub>n-1</sub>** . We'll call the use of this selection table the function **E**. Thus **E(R<sub>n-1</sub>)** has a 32 bit input block, and a 48 bit output block.  

Next in the ***f*** calculation, we XOR the output **E(R<sub>n-1</sub>)** with the key **K<sub>n</sub>**:  

**K<sub>n</sub> + E(R<sub>n-1</sub>)**  

Write the previous result, which is 48 bits, in the form:  

**K<sub>n</sub> + E(R<sub>n-1</sub>) =B<sub>1</sub>B<sub>2</sub>B<sub>3</sub>B<sub>4</sub>B<sub>5</sub>B<sub>6</sub>B<sub>7</sub>B<sub>8</sub>**

We now calculate  

**S<sub>1</sub>(B<sub>1</sub>)S<sub>2</sub>(B<sub>2</sub>)S<sub>3</sub>(B<sub>3</sub>)S<sub>4</sub>(B<sub>4</sub>)S<sub>5</sub>(B<sub>5</sub>)S<sub>6</sub>(B<sub>6</sub>)S<sub>7</sub>(B<sub>7</sub>)S<sub>8</sub>(B<sub>8</sub>)**  

where **S<sub>i</sub>(B<sub>i</sub>)** referres to the output of the **i**-th **S** box.  

If **S<sub>1</sub>** is the function defined in this table and **B** is a block of 6 bits, then **S<sub>1</sub>(B)** is determined as follows: The first and last bits of **B** represent in base 2 a number in the decimal range 0 to 3 (or binary 00 to 11). Let that number be **i**. The middle 4 bits of **B** represent in base 2 a number in the decimal range 0 to 15 (binary 0000 to 1111). Let that number be **j**. Look up in the table the number in the **i**-th row and **j**-th column. It is a number in the range 0 to 15 and is uniquely represented by a 4 bit block. That block is the output **S<sub>1</sub>(B)** of **S<sub>1</sub>** for the input **B**.  

The final stage in the calculation of ***f*** is to do a permutation **P** of the **S**-box output to obtain the final value of ***f***:  

***f* = P(S<sub>1</sub>(B<sub>1</sub>)S<sub>2</sub>(B<sub>2</sub>)S<sub>3</sub>(B<sub>3</sub>)S<sub>4</sub>(B<sub>4</sub>)S<sub>5</sub>(B<sub>5</sub>)S<sub>6</sub>(B<sub>6</sub>)S<sub>7</sub>(B<sub>7</sub>)S<sub>8</sub>(B<sub>8</sub>))** 

In [ ]:
for i in range(len(l_list[:-1])):
    l_list[i+1] = r_list[i]
    e_str = ''.join([r_list[i][idx - 1] for row in E for idx in row])
    xor_out = f'{int(subkeys[i],2) ^ int(e_str,2):0>48b}'
    xor_out_list = [xor_out[idx:idx+6] for idx in range(0, len(xor_out), 6)]
    s_box_output = []
    for idx, gr in enumerate(xor_out_list):
        row = int(gr[0] + gr[-1], 2)
        col = int(gr[1:-1], 2)
        s_box_output.append(f'{s_box[idx][row][col]:0>4b}')
    s_box_ret = ''.join(s_box_output)
    f_ret = ''.join([s_box_ret[idx - 1] for row in P for idx in row])
    r_list[i+1] = f'{int(l_list[i], 2) ^ int(f_ret, 2):0>32b}'
    
    print('E(R{}): {}'.format(i, ' '.join([e_str[idx:idx+6] for idx in range(0, len(e_str), 6)])))
    print('K{}: {}'.format(i+1, ' '.join([subkeys[i][idx:idx+6] for idx in range(0, len(subkeys[i]), 6)])))
    print('A{} = E(R{}) XOR K{}: {}'.format(i+1, i, i+1, ' '.join(xor_out_list)))
    print('S-box: {}'.format(' '.join(s_box_output)))
    print('B{}: {}'.format(i + 1, s_box_ret))
    print('P(B): {})'.format(f_ret))
    print('L{}: {}'.format(i + 1,' '.join([l_list[i+1][idx:idx+4] for idx in range(0, len(l_list[i+1]), 4)])))
    print('R{}: {}'.format(i + 1,' '.join([r_list[i+1][idx:idx+4] for idx in range(0, len(r_list[i+1]), 4)])))
    print('--------------------------------')

At the end of the sixteenth round we have the blocks **L<sub>16</sub>** and **R<sub>16</sub>**. We then reverse the order of the two blocks into the 64-bit block  

**R<sub>16</sub>L<sub>16</sub>**

and apply a final permutation **IP-1** as defined by the **final_per** table.

In [ ]:
cipher = ''.join([(r_list[-1] + l_list[-1])[idx - 1] for row in final_per for idx in row])

print('Ciphertext: {}'.format(hex(int(cipher, 2))[2:].upper()))